In [3]:
import pandas as pd
import pickle
import re
import warnings
import matplotlib.pyplot as plt
import zipfile
import os

warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

EXCEL_FILE_PATH = "SL Property Export NM+RANCHOS+DE+TAOS.xlsx"
MODEL_FILE_PATH = "logreg_classifier.pickle"
VECTORIZER_FILE_PATH = "logreg_vectorizer.pickle"
CLASSIFIED_FILE_PATH = f"Classified/{EXCEL_FILE_PATH} Classified.xlsx"
FILTERED_FILE_PATH = f"Scrubbed/{EXCEL_FILE_PATH} SCRUBBED.xlsx"
ZIP_FILE_PATH = f"Scrubbed/{EXCEL_FILE_PATH} ZIPPED.zip"
summary_file_path = f"Scrubbed/{EXCEL_FILE_PATH} Report.txt"

def load_data(file_path):
    try:
        return pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return pd.DataFrame()

def clean_data(df):
    df.drop_duplicates(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP'], inplace=True)
    df.dropna(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP', 'OWNER_NAME_1'], inplace=True)
    df['Full Name'] = df['OWNER_NAME_1'].str.replace(',', '').str.replace('&', '')
    return df

def load_model_and_vectorizer(model_path, vectorizer_path):
    try:
        with open(model_path, "rb") as model_file:
            model = pickle.load(model_file)
        with open(vectorizer_path, "rb") as vec_file:
            vectorizer = pickle.load(vec_file)
        return model, vectorizer
    except FileNotFoundError as e:
        print(f"File not found: {e.filename}")
        return None, None

def predict_names(df, model, vectorizer):
    if 'OWNER_NAME_1' in df.columns:
        X_test = vectorizer.transform(df['Full Name'])
        df['Prediction'] = model.predict(X_test)
        return df
    else:
        print("The 'OWNER_NAME_1' column is missing from the Excel file.")
        return df

def save_predictions(df, file_path):
    df.to_excel(file_path, index=False)

def filter_company_names(df, human_names):
    df = df[df['OWNER_NAME_1'].isin(human_names)]
    return df

def remove_suspected_company_names(df, keywords):
    pattern = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'
    df = df[~df['OWNER_NAME_1'].str.contains(pattern, case=False, na=False)]
    return df

def visualize_summary_statistics(df_classified):
    stats = {}
    df = load_data(f"Data/{EXCEL_FILE_PATH}")
    # Count initial records
    initial_count = len(df)
    stats['Original Number of Records'] = initial_count

    # Drop missing values and count remaining records
    df_cleaned_missing = df.dropna(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP', 'OWNER_NAME_1'])
    missing_dropped_count = initial_count - len(df_cleaned_missing)
    missing_dropped = (missing_dropped_count / initial_count)
    stats['Missing Data Dropped (%)'] = missing_dropped
    stats['Number of Missing Data'] = missing_dropped_count

    # Drop duplicates and count remaining records (after dropping missing values)
    df_cleaned_duplicates = df_cleaned_missing.drop_duplicates(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP'])
    duplicated_dropped_count = len(df_cleaned_missing) - len(df_cleaned_duplicates)
    duplicated_dropped = (duplicated_dropped_count / len(df_cleaned_missing))
    stats['Duplicated Data Dropped (%)'] = duplicated_dropped
    stats['Number of Duplicated Data'] = duplicated_dropped_count

    # Number of human names remaining
    human_names_remaining = len(df_classified[df_classified['Prediction'] == 'Human Name'])
    stats['Human Names Remaining'] = human_names_remaining

    # Number of company names classified
    company_names_classified = len(df_classified[df_classified['Prediction'] == 'Company Name'])
    stats['Company Names Classified'] = company_names_classified

    # Save summary report to a file
    with open(summary_file_path, "w") as f:
        f.write("---------------------------- Summary Report -----------------------------\n")

        for metric, value in stats.items():
            # Format metric name
            metric_name = metric.replace('_', ' ').title()

            # Format value based on type
            if isinstance(value, float):
                # Display float with one decimal place and '%' if it's between 0 and 1
                if 0 <= value <= 1:
                    formatted_value = f"{value*100:.1f}%"
                else:
                    formatted_value = f"{value:,.0f}"
            else:
                # Display integers with commas
                formatted_value = f"{value:,}"

            f.write(f"{metric_name:<30} {formatted_value:>20}\n")

        f.write("\n-------------------------------------------------------------------------\n")

    print(f"Summary report saved to 'Scrubbed/{EXCEL_FILE_PATH} Statistical Report.txt'")
    return summary_file_path


def zip_files(zip_file_path, files_to_zip):
    with zipfile.ZipFile(zip_file_path, 'w') as zf:
        for file in files_to_zip:
            zf.write(file, os.path.basename(file))
    print(f"Files have been zipped to: {zip_file_path}")


# Main process
df_initial = load_data(f"Data/{EXCEL_FILE_PATH}")
df_cleaned = clean_data(df_initial)

model, vectorizer = load_model_and_vectorizer(MODEL_FILE_PATH, VECTORIZER_FILE_PATH)
if model and vectorizer:
    df_classified = predict_names(df_cleaned, model, vectorizer)
    save_predictions(df_classified[['OWNER_NAME_1', 'Prediction']], CLASSIFIED_FILE_PATH)
    print("Data Classified")
     
    filtered_df = load_data(CLASSIFIED_FILE_PATH)
    human_names = filtered_df[filtered_df['Prediction'] == 'Human Name']['OWNER_NAME_1'].unique().tolist()
    df_filtered = filter_company_names(df_cleaned, human_names)
    save_predictions(df_filtered, FILTERED_FILE_PATH)
    print("Data Filtered and Saved")

    # Generate summary statistics and save report
    summary_file_path = visualize_summary_statistics(df_classified)

    # Zip the filtered file and the summary report
    zip_files(ZIP_FILE_PATH, [FILTERED_FILE_PATH, summary_file_path])


Data Classified
Data Filtered and Saved
Summary report saved to 'Scrubbed/SL Property Export NM+RANCHOS+DE+TAOS.xlsx Statistical Report.txt'
Files have been zipped to: Scrubbed/SL Property Export NM+RANCHOS+DE+TAOS.xlsx ZIPPED.zip


In [4]:
df = load_data(f"Data/{EXCEL_FILE_PATH}")
df_classsified = predict_names(df_cleaned, model, vectorizer)
# Count initial records
initial_count = len(df)
df['Original Number of Records'] = initial_count

# Drop missing values and count remaining records
df_cleaned_missing = df.dropna(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP', 'OWNER_NAME_1'])
missing_dropped_count = initial_count - len(df_cleaned_missing)
missing_dropped = (missing_dropped_count / initial_count)
df['Missing Data Dropped (%)'] = missing_dropped
df['Number of Missing Data'] = missing_dropped_count

print(f"Missing data counter: {missing_dropped_count}, Missing data Percentage: {missing_dropped}")
# Drop duplicates and count remaining records (after dropping missing values)
df_cleaned_duplicates = df_cleaned_missing.drop_duplicates(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP'])
duplicated_dropped_count = len(df_cleaned_missing) - len(df_cleaned_duplicates)
duplicated_dropped = (duplicated_dropped_count / len(df_cleaned_missing))
df['Duplicated Data Dropped (%)'] = duplicated_dropped
df['Number of Duplicated Data'] = duplicated_dropped_count
print(f"Missing data counter: {duplicated_dropped_count}, Missing data Percentage: {duplicated_dropped}")

# Number of human names remaining
human_names_remaining = len(df_classified[df_classsified['Prediction'] == 'Human Name'])
df['Human Names Remaining'] = human_names_remaining

# Number of company names classified
company_names_classified = len(df_classified[df_classified['Prediction'] == 'Company Name'])
df['Company Names Classified'] = company_names_classified

Missing data counter: 4, Missing data Percentage: 0.009009009009009009
Missing data counter: 59, Missing data Percentage: 0.1340909090909091
